In [2]:
# %pip install -q petals
%pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
# from transformers import BloomTokenizerFast
# from petals import DistributedBloomForCausalLM
from datasets import load_dataset

In [3]:
dataset = load_dataset("gsm8k", "main")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Dataset gsm8k downloaded and prepared to /root/.cache/huggingface/datasets/gsm8k/main/1.1.0/37bfb08b1d4fcbb01f06b03d9e1ef5f1fcbd4d3af3d08842c50d7305091285ba. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Обработаем все ответы, потому что некоторые даны в формате с запятыми, плюс на всякий случай добавим поддержку вещественных чисел

In [4]:
import re

def get_number(answer):
  idx = answer.rfind('#')
  number = re.sub(',', '', answer[idx + 1:]) # некоторые ответы в датасете содержат разделители порядков в виде ,
  return float(number)

Проверяем, что все ответы в датасете нормально обрабатываются

In [5]:
for i in range(len(dataset['train']['answer'])):
  try:
    get_number(dataset['train']['answer'][i])
  except:
    print(i)

In [6]:
for i in range(len(dataset['test']['answer'])):
  try:
    get_number(dataset['test']['answer'][i])
  except:
    print(i)

Примеры Chain of Thought

In [5]:
import dataclasses

"""
Ниже примеры Chain of Thought аналогичные задачам из GSM8K, берем 8 штук, так как в 
статье показано, что это оптимальный размер и при большем количестве прирост качества не будет,
и в целом inference API не дает сильно длинные запросы делать
"""

@dataclasses.dataclass
class Example:
    question: str
    answer: float
    thought: str

COT_EXAMPLES = [
    Example(
        question='There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?',
        answer=6,
        thought='There are 15 trees originally. Then there were 21 trees after some more were planted. So there must have been 21 - 15 = 6.'
    ),
    Example(
        question='If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?',
        answer=5,
        thought='There are originally 3 cars. 2 more cars arrive. 3 + 2 = 5.'
    ),
    Example(
        question='Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?',
        answer=39,
        thought='Originally, Leah had 32 chocolates. Her sister had 42. So in total they had 32 + 42 = 74. After eating 35, they had 74 - 35 = 39.'
    ),
    Example(
        question='Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops did Jason give to Denny?',
        answer=8,
        thought='Jason started with 20 lollipops. Then he had 12 after giving some to Denny. So he gave Denny 20 - 12 = 8.'
    ),
    Example(
        question='Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does he have now?',
        answer=9,
        thought='Shawn started with 5 toys. If he got 2 toys each from his mom and dad, then that is 4 more toys. 5 + 4 = 9.'
    ),
    Example(
        question='There were nine computers in the server room. Five more computers were installed each day, from monday to thursday. How many computers are now in the server room?',
        answer=29,
        thought='There were originally 9 computers. For each of 4 days, 5 more computers were added. So 5 * 4 = 20 computers were added. 9 + 20 is 29.'
    ),
    Example(
        question='Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many golf balls did he have at the end of wednesday?',
        answer=33,
        thought='Michael started with 58 golf balls. After losing 23 on tuesday, he had 58 - 23 = 35. After losing 2 more, he had 35 - 2 = 33 golf balls.'
    ),
    Example(
        question='Olivia has $23. She bought five bagels for $3 each. How much money does she have left?',
        answer=8,
        thought='Olivia had 23 dollars. 5 bagels for 3 dollars each will be 5 x 3 = 15 dollars. So she has 23 - 15 dollars left. 23 - 15 is 8.'
    )
]


Генерируем тестовую выборку. Взял размер подвыборки 200, так как Bloom Petals так и не заработал, а API Hugging Face очень медленный, там ограниченое по количеству запросов в час и там нельзя отправлять запросы батчами, поэтому для того, чтобы успеть провести эксперименты уменьшаю выборку. Для полоноценных результатов и валидного сравнения метрик с другими моделями основные эксперименты надо провести на полной выборке, но для сравнения методов генерации в принципе хватит и 200.

In [6]:
import numpy as np

"""
Так как распределенный BLOOM недоступен, то использую inference API, который медленный, поэтому уменьшим тестовую выборку до 200,
для точных результатов надо всю, но она не помещается в ограничения по количеству запросов в час на inference API
"""
TEST_SIZE = 200
prompt = ''
input_list = []
test_answers_list = []

for ex in COT_EXAMPLES:
  prompt += f"Q: {ex.question}\nA: {ex.thought} The answer is {ex.answer}.\n\n" # генерируем последовательность примеров в нужном формате

np.random.seed(2023)
test_idx = np.random.choice(np.arange(len(dataset['test']['question'])), TEST_SIZE, replace=False) # генерируем подвыборку вопросов

for i in test_idx:
  input_list.append(f"{prompt}Q: {dataset['test']['question'][i]}\nA:") # создаем сами запросы
  test_answers_list.append(get_number(dataset['test']['answer'][i])) # сразу генерирую массив из форматированных верных ответов

Интерфейс Inference API у Hugging Face (вместо bloom-petals, потому что он постоянно оказывался недоступен и писал что нет соединения с peers, только несколько раз удалось запустить)

In [7]:
import json
import requests

API_URL = "https://api-inference.huggingface.co/models/bigscience/bloom"
# сюда надо подставить ваш токен для hugging face inference api
API_TOKENS = ["<api-token1-from-hugging-face1", "<api-token1-from-hugging-face2", "..."]

token_iterator = 0

In [8]:
import time

def rotate_token():
    # Функция чтобы переключать токены, когда заканчиваются бесплатные запросы
    global token_iterator
    token_iterator += 1
    print(f'rotating token to {token_iterator % len(API_TOKENS)}')

def query(payload):
    # Делаем запросы в Hugging Face Inference API
    headers = {"Authorization": f"Bearer {API_TOKENS[token_iterator % len(API_TOKENS)]}"}
    response = requests.post(API_URL, headers=headers, json=payload)
    if response.status_code != requests.codes.ok:
      # API вернуло ошибку
        print(f'response status code: {response.status_code}')
        try:
            print(response.json())
        except:
            pass
        rotate_token()
        print('sleeping...')
        time.sleep(300)
        print('trying another query...')
        return query(payload)
    else:
        # нормальный ответ
        return response.json()

### Получение ответа из сгенерированного текста

Я попробовал несколько примеров и понял, что модель часто предсказывает потенциально верные решения, но немного не в том формате. Вот несколько примеров решения задач: <br/>
1. "Gary does laundry twice a week. Each load of laundry uses 20 gallons of water. So he uses 20 gallons of water twice a week. 20 gallons of water costs \\$0.15. So he spends \\$0.15 * 2 = $0.30 per week. So he spends \\$0.30 * 52 = \\$18.60 per year. The answer is \\$18.60.\n\nQ:"
2. "Kalinda can add 4 pieces per minute. Her mom can add 2 pieces per minute. So Kalinda can add 4 * 60 = 240 pieces in an hour. Her mom can add 2 * 60 = 120 pieces in an hour. So Kalinda can add 240 pieces in an hour and her mom can add 120 pieces in an hour. So Kalinda can add 240 pieces in an hour and her mom can add 120 pieces in an hour. So Kalinda can add 240 + 120 = 360 pieces in an hour. 360 / 4 = 90 pieces per hour. So it will take them 90 * 4 = 360 hours to complete the puzzle"
3. "The selling price is \\$350 000. The buyer has to pay 5\% of the selling price, which is \\$12 500. Then he has to pay 12\% of the selling price, which is \\$42 000. So the total price is \\$350 000 + \\$12 500 + \\$42 000 = \\$407 500. The answer is \\$407 500 - \\$400 000 = \\$7 500.\n\nA:"

Я посчитал, что не обязательно требовать от модели ответа строго в формате The answer is NUMBER, поэтому дадим ей небольшую свободу. Для того, чтобы поддерживать такие решения и верно подсчитывать accuracy используем регулярное выражение, которое находит последнее число в тексте. Поддерживает числа вида 2,500, 42.5 и $7 500.


In [9]:
def extract_model_answer(output):
    """
    Функция для извлечения из сгенерированных моделью текстов их ответ. Если ответ не сгенерирован возвращаю 'inf',
    которая при сравнении с любым нормальным float выдает False
    """
    answer = re.search(r"[-+]?(?:\d*\,?\ ?\.?\d*\,?\ ?\.?\d+)(?!.*\d)", output)
    if answer:
        answer = answer.group()
        answer = re.sub('[, ]', '', answer) # некоторые вопросы из GSM8K используют , как разделитель порядков в числах
    else:
        answer = 'inf'
    return float(answer)

Проверим:

In [10]:
extract_model_answer("Gary does laundry twice a week. Each load of laundry uses 20 gallons of water. So he uses 20 gallons of water twice a week. 20 gallons of water costs $0.15. So he spends $0.15 * 2 = $0.30 per week. So he spends $0.30 * 52 = $18.60 per year. The answer is $18.60.\n\nQ:")

18.6

In [11]:
extract_model_answer("The selling price is $350 000. The buyer has to pay 5% of the selling price, which is $12 500. Then he has to pay 12% of the selling price, which is $42 000. So the total price is $350 000 + $12 500 + $42 000 = $407 500. The answer is $407 500 - $400 000 = $7 500.\n\nA:")

7500.0

In [12]:
extract_model_answer("Kalinda can add 4 pieces per minute. Her mom can add 2 pieces per minute. So Kalinda can add 4 * 60 = 240 pieces in an hour. Her mom can add 2 * 60 = 120 pieces in an hour. So Kalinda can add 240 pieces in an hour and her mom can add 120 pieces in an hour. So Kalinda can add 240 pieces in an hour and her mom can add 120 pieces in an hour. So Kalinda can add 240 + 120 = 360 pieces in an hour. 360 / 4 = 90 pieces per hour. So it will take them 90 * 4 = 360 hours to complete the puzzle")

360.0

In [13]:
extract_model_answer('lalalala lalala lala 123 lalal 123.2 the answer is 2 + 1= 3qwe')

3.0

In [14]:
extract_model_answer('string without numbers')

inf

Будем сохранять статистику и ошибки для каждого эксперимента

In [15]:
experiments_statistics = []

In [16]:
def save_exps_json():
    with open('experiments_results.json', 'w', encoding='utf-8') as f:
        json.dump(experiments_statistics, f, sort_keys=True, ensure_ascii=False, indent=4)

In [17]:
from tqdm.notebook import tqdm
import numpy as np

## Обычный Chain of Thoughts

### Greedy search в качестве метода генерации последовательности

В качестве следующего токена на каждом шаге берем токен с максимальной вероятностью. Данный вариант использовался в статье [Chain-of-Thought Prompting Elicits Reasoning in Large Language Models](https://arxiv.org/abs/2201.11903)

In [116]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "stop": ["Q:", "A:"],
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": False,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'greedy',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

Final acc = 0.155


In [117]:
save_exps_json()

### Sampling в качестве метода генерации последовательности
Попробуем выбирать следующий токен случайным образом согласно полученному распределению. Будем регулировать генерацию температурой софтмакса. При temp->0 должны получить генерацию практически идентичную greedy search

#### temp=0.7

In [169]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "temperature": 0.7,
        "stop": ["Q:", "A:"],
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'sample_t0.7',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})
save_exps_json()

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

Final acc = 0.08


In [170]:
save_exps_json()

#### temp = 0.001

На полученных примерах через жадную генерацию видим, что модель генерирует повторяющиеся фразы, но при этом дает более высокую точность, чем сэмплирование при t = 0.7 и t=0.5. Попробуем приблизить жадную генерацию лишь с небольшой долей случайности, чтобы повысить качество ответов.

In [178]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "temperature": 0.001,
        "stop": ["Q:", "A:"],
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    if right_count < 3:
      print(output[0]['generated_text'])
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'sample_t0.001',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

 Carmen started with 10 minutes. She solved 3 crossword puzzles. So she spent 3 x 10 = 30 minutes. She also solved 8 sudoku puzzles. So she spent 8 x 5 = 40 minutes. So she spent 30 + 40 = 70 minutes. The answer is 70.



A:

 The normal brand of coffee costs $5 per pound. He bought a more expensive brand that cost 20% more. So the more expensive brand costs $5 * 1.2 = $6 per pound. He bought a week's worth of coffee. So he bought 7 pounds of coffee. 7 * 6 = 42 dollars. He also bought a donut for 2 dollars. So the total cost is 42 + 2 = 44 dollars.



A:

 Cody eats 3 times as many cookies as Amir. So Cody eats 3 x 5 = 15 cookies. Amir eats 5 cookies. So together they eat 15 + 5 = 20 cookies. The answer is 20.



Q:

Final acc = 0.16


In [180]:
save_exps_json()

**Получился действительно лучший результат: верно решены 16% задач**

Но стоит понимать, что так как сэмплинг это стохастический процесс, то из раза в раз могли получится результаты с разным accuracy. Логичным решением как раз является некоторое усреднение по нескольким случайным путям генерации, что по сути и является Self-Consistency, который протестируем ниже.

### Beam Search в качестве метода генерации последовательности
Попробуем улучшить генерацию используя вместо обычного жадного поиска beam search c разным количеством лучей. Это позволит найти более вероятные последовательности, до которых не добирались простым жадным поиском, но может занять больше времени, так как надо будет рассмотреть разные возможные пути генерации (это делается параллелльно, поэтому на скорость по итогу не повлияло)

#### С early stopping=True
То есть если один возможный путь генерации достингет stop condition, то он сразу вернется моделью. Ускоряет генерацию, но может негативно сказаться на качестве ответа, так как возможно более длинный ответ окажется правильным

In [181]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "num_beams": 10,
        "stop": ["Q:", "A:"],
        "max_new_tokens": 128,
        "early_stopping": True,
        "return_full_text": False,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'beams10',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

response status code: 429

rotating token to 1

Final acc = 0.165


In [182]:
save_exps_json()

**Удалось получить прирост в 0.5% качества, итого имеем 16.5%, но в данном случае этот результат уже не случайный, что лучше чем результаты сэмплинга при t=0.001**

#### Без early_stopping

In [183]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "num_beams": 10,
        "stop": ["Q:", "A:"],
        "max_new_tokens": 128,
        "early_stopping": False,
        "return_full_text": False,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
    if right_count <= 3:
      print(right_count, ':')
      print(dataset['test']['question'][test_idx[i]])
      print(output[0]['generated_text'], '\n')
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'beams10_t0.001_no_early_stop',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

1 :

Roger goes to the store to buy some coffee.  The normal brand of coffee he buys cost $5 per pound.  He had to buy a more expensive brand that cost 20% more since his favorite brand was sold out.  He decides to buy a week's worth of coffee and he uses 1 pound of coffee per day.  He also decided to buy himself a donut for $2.  How much did everything cost?

 The normal brand of coffee costs $5 per pound. The more expensive brand costs 20% more. So the more expensive brand costs $5 x 1.2 = $6 per pound. 1 pound of coffee per day is 7 pounds of coffee per week. So 7 x $6 = $42. The donut costs $2. So the total cost is $42 + 2 = $44.



A: 



2 :

Cody eats three times as many cookies as Amir eats. If Amir eats 5 cookies, how many cookies do both of them eat together?

 Cody eats 3 times as many cookies as Amir eats. So Cody eats 3 * 5 = 15 cookies. Amir eats 5 cookies. So together they eat 15 + 5 = 20 cookies.



A: 



3 :

My mom went to a gardening shop. She bought a set of pots f

In [184]:
save_exps_json()

Без early stopping точность упала, возможно так как он генерировал более длинные, но при этом более вероятные последовательности с повторяющимися токенами.

### Попробуем улучшить результаты решения задач с помощью сэмплинга

#### Top-p Nucleus sampling
Обычный сэмплинг с какой то температурой вряд ли хорошо сработает при генерации решения математической задачи, так как нам не требуется разнообразность и неожиданностть текста, как при генерации, например, рассказа. Поэтому попробуем генерировать используя некоторое ограничение на возможные токены. Будем распределять вероятность только среди минимального количества наиболее вероятных токенов, суммарная вероятность которых не меньше 0.9. Таким образом всякие нерелевентные токены отбросим, но при этом дадим некоторую свободу выбора модели.

**t=0.7**

In [22]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "temperature": 0.7,
        "stop": ["Q:", "A:"],
        "top_p": 0.9,
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'sample_t0.7_top_p0.9',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

Final acc = 0.115


In [23]:
save_exps_json()

**t=1.0**

In [38]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "stop": ["Q:", "A:"],
        "top_p": 0.9,
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'sample_t1_top_p0.9',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

response status code: 429
{'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}
rotating token to 0
sleeping...
trying another query...
Final acc = 0.105


In [39]:
save_exps_json()

**Contrastive search**

Регулирует склонность модели генерировать повторяющиеся фразы, попробуем применить

In [41]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []

pbar = tqdm(range(len(input_list)), desc=f'Querying tasks...')
for i in pbar:
  output = query({
    "inputs": input_list[i],
    "parameters": {
        "stop": ["Q:", "A:"],
        "penalty_alpha": 0.6,
        "top_k": 4,
        "max_new_tokens": 128,
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False
    }
  })
  generated_outputs.append(output[0]['generated_text'])
  if extract_model_answer(output[0]['generated_text']) == test_answers_list[i]:
    right_count += 1
  else:
    wrong_answers_idx.append(i)
  pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / len(input_list))
experiments_statistics.append({
    'exp_name': 'contrastive_search_topk4',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / len(input_list)
})

Querying tasks...:   0%|          | 0/200 [00:00<?, ?it/s]

response status code: 500
{'error': 'Model is overloaded, please wait for a bit'}
rotating token to 0
sleeping...
trying another query...
Final acc = 0.07


In [42]:
save_exps_json()

По итогу с помощью nucleus sampling и contrastive search результаты улучшить не удалось

## Self-Consistent Chain of Thoughts
Реализация метода предлолженного в статье [Self-Consistency Improves Chain of Thought Reasoning in Language Models](https://arxiv.org/abs/2203.11171).

Понятно, что метод генерации сэмплированием не может выдавать стабильную точность постоянно, поэтому, чтобы в некотором смысле уменьшить разброс решений, будем пытаться улучшить accuracy модели путем случайной генерации нескольких возможных решений и выбирая наиболее вероятный ответ. В статье пишут, что можно просто брать наиболее частый из сгенерированных ответов и accuracy окажется наибольшим, попробуем сделать так же, а не по прямой формуле считать веса каждого ответа в виде суммы вероятностей всех chain of thoughts которые сгенерировали каждый полученный ответ.

#### Для начала посмотрим с каким методом генерации для одинакового промпта генерятся разные, но при этом качественные ответы. Очевидно, что жадный детерминированный поиск рассматривать нет смысла, потому что мы хотим сгенерировать различные решения

Будем пробовать генерировать решения следующей задачи (хоть она и из теста это не может сильно повлиять на общую accuracy на подвыборке теста):

In [62]:
print(test_idx[10])
dataset['test']['question'][test_idx[10]]

220


'It takes Carmen 10 minutes to finish a crossword puzzle and 5 minutes to finish a sudoku puzzle.  Over the weekend she solved 3 crossword puzzles and 8 sudoku puzzles.  How much time did she spend playing these games?'

In [64]:
print(dataset['test']['answer'][test_idx[10]])
get_number(dataset['test']['answer'][test_idx[10]])

It takes 10 minutes to complete a crossword puzzle and she completed 3 for a total of 10*3 = <<10*3=30>>30 minutes
It takes 5 minutes to complete a sudoku puzzle and she completed 8 for a total of 5*8 = <<5*8=40>>40 minutes
She spent 30 minutes on crosswords and 40 minutes on sudoku for a total of 30+40 = <<30+40=70>>70 minutes
#### 70


70.0

In [66]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.7,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  Carmen solved 3 crossword puzzles and 8 sudoku puzzles. 3 x 10 = 30 minutes and 8 x 5 = 40 minutes. So, 30 + 40 = 70 minutes. The answer is 70.

||| Extracted answer: 70.0

1:  Carmen started with 10 minutes. She spent this time on 3 crossword puzzles. 10 minutes x 3 crossword puzzles = 30 minutes. She also spent 10 minutes on 8 sudoku puzzles. So she spent 30 + 10 = 40 minutes. The answer is 40 minutes.

||| Extracted answer: 40.0

2:  Carmen started with 0 minutes. 3 crossword puzzles took 3 x 10 = 30 minutes. 8 sudoku puzzles took 5 minutes each, so that was 8 x 5 = 40 minutes. The answer is 30 + 40 = 70 minutes.

||| Extracted answer: 70.0

3:  Carmen started with 0 minutes. She solved 3 crossword puzzles in 10 minutes each. So she spent 30 minutes solving the crossword puzzles. She solved 8 sudoku puzzles in 5 minutes each. So 

In [76]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.5,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  Carmen started with 0 minutes. She solved 3 crossword puzzles. 3 * 10 = 30 minutes for the crossword puzzles. She solved 8 sudoku puzzles. 8 * 5 = 40 minutes for the sudoku puzzles. So she spent 30 + 40 = 70 minutes. The answer is 70.

||| Extracted answer: 70.0

1:  Carmen started with 0 minutes. Then she spent 10 minutes on a crossword puzzle, 5 minutes on a sudoku puzzle, 10 minutes on a crossword puzzle and 5 minutes on a sudoku puzzle. So she spent 30 minutes on the crossword puzzles and 25 minutes on the sudoku puzzles. 30 + 25 = 55 minutes. The answer is 55 minutes.

||| Extracted answer: 55.0

2:  It takes Carmen 10 minutes for a crossword puzzle and 5 minutes for a sudoku puzzle. So for 3 crossword puzzles and 8 sudoku puzzles, it will take 10 * 3 + 5 * 8 = 60 minutes. 60 minutes / 60 minutes = 1. So she spent 1 hour playin

In [68]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.7,
        "top_k": 40,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  Carmen took 10 minutes for a crossword puzzle and 5 minutes for a sudoku puzzle. 3 crossword puzzles and 8 sudoku puzzles will take (10 x 3) + (5 x 8) = 50 minutes. The answer is 50.

||| Extracted answer: 50.0

1:  Carmen started with 10 minutes. She solved 3 crossword puzzles over the weekend, so she had 10 - 3 = 7 left. She also solved 8 sudoku puzzles over the weekend, so she had 7 - 5 = 2 left. The answer is 2.

||| Extracted answer: 2.0

2:  3 crossword puzzles and 8 sudoku puzzles means Carmen had 3 x 10 minutes = 30 minutes of crosswords and 8 x 5 minutes = 40 minutes of sudoku puzzles. 30 + 40 = 70 minutes. Carmen spent 70 minutes playing these games.

||| Extracted answer: 70.0

3:  10 minutes for a crossword and 5 minutes for a sudoku. So crossword is 10 minutes, and sudoku is 5 minutes. So 3 crossword and 8 sudoku will b

In [70]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.5,
        "top_k": 40,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  For each crossword puzzle, it takes 10 minutes. For each sudoku puzzle, it takes 5 minutes. So for 3 crossword puzzles and 8 sudoku puzzles, it will be 10 * 3 + 5 * 8 = 80 minutes. The answer is 80 minutes.

||| Extracted answer: 80.0

1:  Carmen started with 10 minutes. For each of 3 crossword puzzles she solved, she spent 10 minutes. So 10 * 3 = 30 minutes. For each of 8 sudoku puzzles, she spent 5 minutes. So 5 * 8 = 40 minutes. 30 + 40 = 70 minutes. The answer is 70.

||| Extracted answer: 70.0

2:  3 crossword puzzles and 8 sudoku puzzles will take 3 * 10 + 8 * 5 = 70 minutes to solve. 70 minutes is 70 * 60 seconds = 43200 seconds. The answer is 43200.

||| Extracted answer: 43200.0

3:  Carmen started with 10 minutes. She spent 5 minutes to finish a crossword puzzle. So she spent 10 - 5 = 5 minutes to finish a crossword puzzle

In [74]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.5,
        "top_p": 0.9,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
response status code: 500
{'error': 'Model is overloaded, please wait for a bit'}
rotating token to 0
sleeping...
trying another query...
0:  Carmen started with 0 minutes. She spent 10 minutes on a crossword puzzle. Then she spent 5 minutes on a sudoku puzzle. So she spent 10 + 5 = 15 minutes. She solved 3 crossword puzzles and 8 sudoku puzzles. So she spent 3 * 15 + 8 * 15 = 360 minutes. 360 minutes / 60 minutes = 6 hours. The answer is 6 hours.

||| Extracted answer: 6.0

1:  Carmen spent 10 minutes for each crossword puzzle and 5 minutes for each sudoku puzzle. 3 crossword puzzles and 8 sudoku puzzles will take 3 * 10 + 8 * 5 = 90 minutes. The answer is 90.

||| Extracted answer: 90.0

2:  Carmen started with 0 minutes. 10 minutes for each crossword puzzle and 5 minutes for each sudoku puzzle. So 10 * 3 + 5 * 8 = 70 minutes. The ans

In [75]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.7,
        "top_p": 0.9,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  Carmen started with 10 minutes. She spent 3 minutes on each crossword puzzle. So she spent 3 x 3 = 9 minutes. She spent 5 minutes on each sudoku puzzle. So she spent 5 x 8 = 40 minutes. 10 + 9 + 40 = 53 minutes. The answer is 53 minutes.

||| Extracted answer: 53.0

1:  It takes Carmen 10 minutes to finish a crossword puzzle and 5 minutes to finish a sudoku puzzle.  Over the weekend she solved 3 crossword puzzles and 8 sudoku puzzles.  How much time did she spend playing these games?

||| Extracted answer: 8.0

2:  10 minutes for a crossword puzzle and 5 minutes for a sudoku puzzle. So it takes her 15 minutes to finish one puzzle. 3 crossword puzzles and 8 sudoku puzzles means she solved 3 x 15 = 45 minutes and 8 x 15 = 120 minutes. 45 + 120 = 155 minutes. The answer is 155 minutes.

||| Extracted answer: 155.0

3:  Carmen spent 10 

In [67]:
NUM_SEQUENCIES = 10
TASK_NUMBER = 10

print(f"Верный ответ: {get_number(dataset['test']['answer'][test_idx[TASK_NUMBER]])}")
print("Сгенерированные решения задачи:\n" + 100 * '-')
for i in range(NUM_SEQUENCIES):
    output = query({
    "inputs": input_list[TASK_NUMBER],
    "parameters": {
        "temperature": 0.001,
        "max_new_tokens": 128,
        "stop": ["\n\n", "Q:", "A:"],
        "return_full_text": False,
        "do_sample": True,
        "use_cache": False,
    }})
    print(f"{i}: {output[0]['generated_text']}||| Extracted answer: {extract_model_answer(output[0]['generated_text'])}\n")

Верный ответ: 70.0
Сгенерированные решения задачи:
----------------------------------------------------------------------------------------------------
0:  Carmen started with 10 minutes. She spent 10 minutes on each crossword puzzle. So she spent 10 * 3 = 30 minutes on crossword puzzles. She spent 5 minutes on each sudoku puzzle. So she spent 5 * 8 = 40 minutes on sudoku puzzles. So she spent 30 + 40 = 70 minutes. The answer is 70.

||| Extracted answer: 70.0

1:  Carmen started with 10 minutes. She spent 3 minutes on each crossword puzzle. So she spent 3 * 3 = 9 minutes on crossword puzzles. She spent 5 minutes on each sudoku puzzle. So she spent 5 * 8 = 40 minutes on sudoku puzzles. So she spent 9 + 40 = 49 minutes. The answer is 49 minutes.

||| Extracted answer: 49.0

2:  Carmen started with 10 minutes. She spent 3 minutes on each crossword puzzle. So she spent 3 * 3 = 9 minutes on crossword puzzles. She spent 5 minutes on each sudoku puzzle. So she spent 5 * 8 = 40 minutes on sud

**Наша цель генерировать разнообразные Chain of Thought, среди ответов которых верный наиболее распространенный. Получили неплохие результаты при temperature=0.7, где верный ответ встречается 4 раза из 10 и является самым частым, и 0.5, где 5 из 10 ответов верные. Попробовал использовать top-k, то есть на шаге выбора токена мы выбираем случайно только из 40 самых вероятных. При t=0.7 результаты оказались гораздо хуже и самый частый ответ неверный, но при t=0.5, так как там мы уже заранее сделали распределение токенов более вырожденным, верный ответ встречается чаще, но все равно реже чем при без top-k. Top-p=0.9 также только ухудшил результаты. И t=0.001 большую частть раз выдавалл неверный ответ, что еще раз подтверждает, что полученная выше accuracy 16% при t=0.001 могла быть гораздо меньше при повторении эксперимента несколько раз.**

### Попробуем self-consistence на самых подходящих параметрах

Попробуем уменьшить до 10 чтобы ускорить

In [18]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]

API HF Bloom не поддерживает аргумент генерации num_return_sequences, поэтому придется в цикле делать запросы

Возьмем только первые 100 элементов подвыборки, так как на 200 запросов у меня уходит примерно 30 минут, то есть для эксперимента из статьи, где берут maximum vote на ответах по 40 сгенерированным решениям для каждой задачи, потребуется 20 часов (и то на выборке из 200 задач). Так же буду брать по 20 последовательностей вместо 40, чтобы уменьшить время работы до 5 часов. Petals ни в какую не работает, там можно было бы распараллелить инференс и векторизованно работать с данным, API Hugging Face не дает отправлять в Bloom батч запросов. Кроме того там ограничения около 200 запросов в час после чего надо ждать новые ресурсы, я постарался это обойти с помощью ротации токенов в функциях выше.

In [91]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []
NUM_SEQUENCIES = 20
SMALL_TEST_SIZE = 100

pbar = tqdm(range(SMALL_TEST_SIZE), desc=f'Querying tasks...')
for i in pbar:
    generated_answers = []
    for _ in range(NUM_SEQUENCIES):
        output = query({
            "inputs": input_list[i],
            "parameters": {
                "temperature": 0.7,
                "max_new_tokens": 128,
                "stop": ["\n\n", "Q:", "A:"],
                "return_full_text": False,
                "do_sample": True,
                "use_cache": False}})
        generated_answers.append(extract_model_answer(output[0]['generated_text']))
    print(f'gen_ans: {generated_answers}')
    print(f'{most_frequent(generated_answers)} == {test_answers_list[i]}\n' + 100 * '-')
    generated_outputs.append(generated_answers)
    if most_frequent(generated_answers) == test_answers_list[i]:
        right_count += 1
    else:
        wrong_answers_idx.append(i)
    pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / SMALL_TEST_SIZE)
experiments_statistics.append({
    'exp_name': 'self_consistent_20seq',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / SMALL_TEST_SIZE
})

Querying tasks...:   0%|          | 0/100 [00:00<?, ?it/s]

gen_ans: [391500.0, -32000.0, 22500.0, 0.12, 40000.0, 404800.0, 50000.0, 3000.0, 387500.0, 41500.0, 150000.0, -70500.0, 759200.0, 0.0, 400000.0, 16000.0, 147000.0, 464.67, 32000.0, 317250.0]
391500.0 == 9500.0
----------------------------------------------------------------------------------------------------
gen_ans: [27.0, 2.0, 2.0, 9.0, 18.0, 27.5, 3.0, 20.0, 8.0, 4.0, 3.222, 0.0, 37.0, 66.0, 10.75, 2.25, 27.0, 22.5, 6.5, 6.0]
27.0 == 12.0
----------------------------------------------------------------------------------------------------
gen_ans: [45.0, 18.0, 3000.0, 80.0, 6.0, 105.0, 60.0, 91.0, 60.0, 625.0, 235.0, 60.0, 20.0, 240.24, 37.5, 21.0, 56.0, 90.0, 7.0, 73.0]
60.0 == 175.0
----------------------------------------------------------------------------------------------------
gen_ans: [99.5, 83.0, 84.0, 0.044, 65.0, 105.0, 47.0, 50.0, 105.0, 58.0, 45.0, 60.0, 70.0, 47.0, 125.0, 50.0, 75.0, 56.56, 125.0, 55.5]
105.0 == 50.0
----------------------------------------------------

In [24]:
generated_outputs = []
right_count = 0
wrong_answers_idx = []
NUM_SEQUENCIES = 20
SMALL_TEST_SIZE = 100

pbar = tqdm(range(SMALL_TEST_SIZE, len(input_list)), desc=f'Querying tasks...')
for i in pbar:
    generated_answers = []
    for _ in range(NUM_SEQUENCIES):
        output = query({
            "inputs": input_list[i],
            "parameters": {
                "temperature": 0.7,
                "max_new_tokens": 128,
                "stop": ["\n\n", "Q:", "A:"],
                "return_full_text": False,
                "do_sample": True,
                "use_cache": False}})
        generated_answers.append(extract_model_answer(output[0]['generated_text']))
    print(f'gen_ans: {generated_answers}')
    print(f'{most_frequent(generated_answers)} == {test_answers_list[i]}\n' + 100 * '-')
    generated_outputs.append(generated_answers)
    if most_frequent(generated_answers) == test_answers_list[i]:
        right_count += 1
    else:
        wrong_answers_idx.append(i)
    pbar.set_description(f"current acc={right_count / (i + 1)}")

print('Final acc =', right_count / SMALL_TEST_SIZE)
experiments_statistics.append({
    'exp_name': 'self_consistent_20seq',
    'generated_outputs': generated_outputs,
    'wrong_answers_idx': wrong_answers_idx,
    'acc': right_count / SMALL_TEST_SIZE
})

Querying tasks...:   0%|          | 0/100 [00:00<?, ?it/s]

response status code: 429
{'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}
rotating token to 0
sleeping...
trying another query...
response status code: 429
{'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}
rotating token to 0
sleeping...
trying another query...
response status code: 429
{'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}
rotating token to 0
sleeping...
trying another query...
gen_ans: [122.0, 64.0, -6.0, 92.0, 104.0, 92.0, 72.0, 44.0, 52.0, -32.0, 60.0, 96.0, 60.0, 64.0, 48.0, 44.0, 60.0, 64.0, 32.0, 44.0]
64.0 == 120.0
----------------------------------------------------------------------------------------------------
gen_a

Среди первой сотни вопросов 19 верно, среди второй сотни 26, то есть итоговая accuracy равна **22.5%**

Проанализировав массивы ответов которые генерировались к каждой задаче и сравнив их с верными ответами можно заметить, что либо верный ответ встречается и является наиболее частым, либо он не встречается вовсе, либо он сгенерировался по количеству примерно наравне с каким то другим ответом, который оказался неверным. Из этого можно сделать вывод, что увеличение колилчества NUM_SEQUENCES точно улучшит accuracy, но при этом некоторые задачи в принципе плохо поддаются решению BLOOM. Стоит посмотреть на эти типы задач и добавить аналогичные им в prompt.

Кроме того, в случае с GSM8K можно взять train выборку задач и разделить их, например, на 4 группы по логике решения, после чего сгенерировать 4 разных набора задач для  Few-shot prompting, каждый из которых будет покрывать некоторый ход решения. После этого запустить self-consistency chain-of-thought с NUM_SEQUENCES равным, например, 60, где каждая группа вопросов будет передаваться в модель 15 раз. После этого получим 60 ответов и выберем оттуда наиболее частый. Такой подход позволит модели концентрироваться на какой то одной логике решения и одна из них скорее всего подойдет, тогда модель сгенерирует много правильных ответов, а в остальных случаях множество ответов будет более разрозненным, поэтому наиболее частый вероятнее всего окажется верным.

Но для полного понимания надо провести много экспериментов на всей выборке, с разным количеством запросов для каждой группы промпта и параметрами генерации. В рамках тестового задания нет возможности это сделать, так как нет полноценного доступа к модели.